In [1]:
import pyvista as pv
import vtk

# 直接读取VTK文件
mesh = pv.read("./liggghts_run_999600.vtk")

# 查看基本信息
print(f"点数: {mesh.n_points}")
print(f"单元数: {mesh.n_cells}")
print(f"边界: {mesh.bounds}")

# 访问点坐标
points = mesh.points
print(f"前5个点坐标:\n{points[:5]}")

点数: 9240
单元数: 9240
边界: BoundsTuple(x_min=0.0005994029925204813, x_max=0.04260079935193062, y_min=0.0005989120109006763, y_max=0.009400660172104836, z_min=0.0005995350074954331, z_max=0.053586799651384354)
前5个点坐标:
[[0.00060038 0.00060267 0.00060137]
 [0.00231073 0.0005998  0.00060006]
 [0.0036888  0.00060006 0.00060731]
 [0.00488955 0.00060334 0.00060332]
 [0.00608727 0.00071655 0.00060136]]


In [5]:
# 访问点数据和单元数据
if mesh.point_data:
    print("点数据字段:", mesh.point_data.keys())
if mesh.cell_data:
    print("单元数据字段:", mesh.cell_data.keys())

# 可视化
# mesh.plot(show_edges=True)

点数据字段: ['id', 'type', 'v', 'f', 'radius', 'omega']


# 转换数据为CSV文件

In [4]:
import pandas as pd

In [ ]:

N = 50  # 选取前50个粒子
mass = 1.0  # 如果不方便估算体积密度，可以先设常数

# 坐标
x, y, z = points[:N, 0], points[:N, 1], points[:N, 2]

# 速度
vx, vy, vz = mesh.point_data['v'][:N].T

# 力
fx, fy, fz = mesh.point_data['f'][:N].T

# 加速度 a = F/m
ax, ay, az = fx / mass, fy / mass, fz / mass

# 打包 DataFrame（PINN训练数据）
df = pd.DataFrame({
    'x': x, 'y': y, 'z': z,
    'vx': vx, 'vy': vy, 'vz': vz,
    'ax': ax, 'ay': ay, 'az': az
})
df.to_csv("pinn_dataset.csv", index=False)

# PINN最小示例

In [1]:
import pandas as pd
import torch
import torch.nn as nn
import torch.autograd as autograd

# === 1. 读取数据 ===
df = pd.read_csv("pinn_dataset.csv")
X = torch.tensor(df[["x", "y", "z"]].values, dtype=torch.float32, requires_grad=True)
V = torch.tensor(df[["vx", "vy", "vz"]].values, dtype=torch.float32)
A = torch.tensor(df[["ax", "ay", "az"]].values, dtype=torch.float32)

# === 2. 构建PINN网络 ===
class PINN(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(3, 64), nn.Tanh(),
            nn.Linear(64, 64), nn.Tanh(),
            nn.Linear(64, 3)
        )
    def forward(self, x):
        return self.net(x)

model = PINN()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
loss_fn = nn.MSELoss()

# === 3. 训练 ===
for epoch in range(1000):
    optimizer.zero_grad()
    v_pred = model(X)
    loss_data = loss_fn(v_pred, V)

    grads = autograd.grad(
        outputs=v_pred, inputs=X,
        grad_outputs=torch.ones_like(v_pred),
        create_graph=True, only_inputs=True
    )[0]

    loss_phys = loss_fn(grads, A)
    loss = loss_data + loss_phys
    loss.backward()
    optimizer.step()

    if epoch % 100 == 0:
        print(f"[{epoch}] total={loss.item():.2e}, data={loss_data.item():.2e}, physics={loss_phys.item():.2e}")

[0] total=2.73e-02, data=1.95e-02, physics=7.81e-03
[100] total=2.16e-05, data=2.14e-05, physics=1.25e-07
[200] total=2.14e-05, data=2.13e-05, physics=8.89e-08
[300] total=2.13e-05, data=2.12e-05, physics=8.86e-08
[400] total=2.12e-05, data=2.11e-05, physics=8.81e-08
[500] total=2.11e-05, data=2.10e-05, physics=8.76e-08
[600] total=2.10e-05, data=2.09e-05, physics=8.70e-08
[700] total=2.09e-05, data=2.08e-05, physics=8.64e-08
[800] total=2.08e-05, data=2.07e-05, physics=8.57e-08
[900] total=2.07e-05, data=2.06e-05, physics=8.50e-08
